In [35]:
import pandas as pd
import json

import requests
from bs4 import BeautifulSoup

import gc
import time
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [36]:
cookies = {
    'ezoadgid_114948': '-1',
    'ezoref_114948': '',
    'ezosuibasgeneris-1': 'a118d960-00f1-4f29-7886-bd88aff9e427',
    'ezoab_114948': 'mod35-c',
    'ezovid_114948': '534297242',
    'lp_114948': 'https://techrakyat.com/',
    'ezovuuid_114948': 'ebbfc0ed-200a-48ca-4ed8-fc37cde2ed7a',
    'ezds': 'ffid%3D1%2Cw%3D1368%2Ch%3D912',
    '_ga': 'GA1.1.1854117891.1694326492',
    '__qca': 'P0-1969678871-1694326498631',
    'ezux_ifep_114948': 'true',
    '_pbjs_userid_consent_data': '3524755945110770',
    'cnx_userId': 'ea3f7c275d3c41e195be89513380856a',
    '_cc_id': '2eb1503300a705422b12f7957f2e9a35',
    'panoramaId_expiry': '1694931329921',
    'panoramaId': '127a715a7ca55b5663be2b1bcf6616d539384e06c3902f002688e78798baabbc',
    'panoramaIdType': 'panoIndiv',
    'ezouspvh': '140',
    'ezouspvv': '0',
    '__gads': 'ID=e10681e1634ad889:T=1694326526:RT=1694328131:S=ALNI_Mb75SVYyM5fCdJq0rB-MfKweKF9SQ',
    '__gpi': 'UID=00000c40799eb4ae:T=1694326526:RT=1694328131:S=ALNI_MbmGjgllIkVUgQeAWxEBPf5oQquJA',
    'ezvignetteviewed': 'true',
    'ezepvv': '0',
    'ezouspva': '0',
    'ezohw': 'w%3D1368%2Ch%3D774',
    '_pubcid': '6e55175a-06b7-4621-8467-b3b963ba5c2d',
    'cto_bundle': 'nGB5B19wVlZNTDJ5Vk96ak96M3BpSU9HMFdOS0tmQTRWSnp3eU5Xa2NMTHR5OEY2cXdXeHdkWWw0NEVPV2tSUTBhWUlKQ1ZtVjlseEFvdFdnRmd2cDNzelROZmhZTDFyb0hpNHY2M3k2Q2dQZ2JEaXZhUDg0MHVJQ2dxbjk5WFZqcDJBYXE5YWZmbEMwdFVQWHZRY1djMUlnJTJGa21MVkg1SFU5azY1SWV2YXNZc0tHVTBmZnVmaHA5NFYlMkJNM2xUWkUlMkJxb2ZSU0VmbHFVZXJKUTJRTjc5ZjlTazlWaGdnJTJGWHV6YiUyQjFLdXl1QzlyNFElMkJESTVRWW5qdEppbm92a081Sk1oOFpnR2ZyM0FsZ1ZvREJFaW12a3RWbzRRQSUzRCUzRA',
    'cto_bidid': '1T8hv180NlBMVVJUclRxeHJmZ1VKTTJyeDQ3aTVtbFhWMTU0a0VHVkZweFp2aEhwYUNEQWJLY0VWMTdobkR2Vm82Slo1MGJOQiUyQjRjMlJyUlBpbnJ0ZnFsbVVJTW9kVUx3QmZsamc3RkJiZ2hEcVdQVk01bUpBMkx6RDNvdFplMDhTdUx0',
    'active_template::114948': 'pub_site.1694328549',
    'ezopvc_114948': '48',
    'ezovuuidtime_114948': '1694328549',
    'ezux_lpl_114948': '1694328550682|52b690fe-abbf-4a1e-6f87-df360cdd8c61|false',
    '_ga_KJN1ZJ79QS': 'GS1.1.1694326492.1.1.1694328556.0.0.0',
    'ezux_et_114948': '712',
    'ezux_tos_114948': '1933',
}

headers = {
    'authority': 'techrakyat.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    # 'cookie': 'ezoadgid_114948=-1; ezoref_114948=; ezosuibasgeneris-1=a118d960-00f1-4f29-7886-bd88aff9e427; ezoab_114948=mod35-c; ezovid_114948=534297242; lp_114948=https://techrakyat.com/; ezovuuid_114948=ebbfc0ed-200a-48ca-4ed8-fc37cde2ed7a; ezds=ffid%3D1%2Cw%3D1368%2Ch%3D912; _ga=GA1.1.1854117891.1694326492; __qca=P0-1969678871-1694326498631; ezux_ifep_114948=true; _pbjs_userid_consent_data=3524755945110770; cnx_userId=ea3f7c275d3c41e195be89513380856a; _cc_id=2eb1503300a705422b12f7957f2e9a35; panoramaId_expiry=1694931329921; panoramaId=127a715a7ca55b5663be2b1bcf6616d539384e06c3902f002688e78798baabbc; panoramaIdType=panoIndiv; ezouspvh=140; ezouspvv=0; __gads=ID=e10681e1634ad889:T=1694326526:RT=1694328131:S=ALNI_Mb75SVYyM5fCdJq0rB-MfKweKF9SQ; __gpi=UID=00000c40799eb4ae:T=1694326526:RT=1694328131:S=ALNI_MbmGjgllIkVUgQeAWxEBPf5oQquJA; ezvignetteviewed=true; ezepvv=0; ezouspva=0; ezohw=w%3D1368%2Ch%3D774; _pubcid=6e55175a-06b7-4621-8467-b3b963ba5c2d; cto_bundle=nGB5B19wVlZNTDJ5Vk96ak96M3BpSU9HMFdOS0tmQTRWSnp3eU5Xa2NMTHR5OEY2cXdXeHdkWWw0NEVPV2tSUTBhWUlKQ1ZtVjlseEFvdFdnRmd2cDNzelROZmhZTDFyb0hpNHY2M3k2Q2dQZ2JEaXZhUDg0MHVJQ2dxbjk5WFZqcDJBYXE5YWZmbEMwdFVQWHZRY1djMUlnJTJGa21MVkg1SFU5azY1SWV2YXNZc0tHVTBmZnVmaHA5NFYlMkJNM2xUWkUlMkJxb2ZSU0VmbHFVZXJKUTJRTjc5ZjlTazlWaGdnJTJGWHV6YiUyQjFLdXl1QzlyNFElMkJESTVRWW5qdEppbm92a081Sk1oOFpnR2ZyM0FsZ1ZvREJFaW12a3RWbzRRQSUzRCUzRA; cto_bidid=1T8hv180NlBMVVJUclRxeHJmZ1VKTTJyeDQ3aTVtbFhWMTU0a0VHVkZweFp2aEhwYUNEQWJLY0VWMTdobkR2Vm82Slo1MGJOQiUyQjRjMlJyUlBpbnJ0ZnFsbVVJTW9kVUx3QmZsamc3RkJiZ2hEcVdQVk01bUpBMkx6RDNvdFplMDhTdUx0; active_template::114948=pub_site.1694328549; ezopvc_114948=48; ezovuuidtime_114948=1694328549; ezux_lpl_114948=1694328550682|52b690fe-abbf-4a1e-6f87-df360cdd8c61|false; _ga_KJN1ZJ79QS=GS1.1.1694326492.1.1.1694328556.0.0.0; ezux_et_114948=712; ezux_tos_114948=1933',
    'referer': 'https://techrakyat.com/tips/',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Google Chrome";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
}

### 1.0 Get links for subsections in the *Reviews* section.

In [37]:
# list down the sections of the website we'd like to scrape.
review_subsections = [
    "home-appliances/",
    "kitchen-appliances/",
    "electronics/",
    "broadband-telco/",
]

In [38]:
to_add = []
for sections in review_subsections:
    url = f"https://techrakyat.com/{sections}"
    response = requests.get(url, cookies=cookies, headers=headers)

    soup = BeautifulSoup(response.text, "lxml")

    block = soup.find('div',attrs = {"class":"elementor-inner"})
    
    for link in block.find_all('h2', attrs = {"class":"elementor-heading-title elementor-size-default"}):
        try:
            href = link.find('a').get('href')
            required_section = href.split(".")[-1][4:]
            to_add.append(required_section)
        except:
            print(f"{sections.title()} doesn't have subsections/tags. Need to manually add.")
            pass

Broadband-Telco/ doesn't have subsections/tags. Need to manually add.
Broadband-Telco/ doesn't have subsections/tags. Need to manually add.


In [39]:
to_add

['tag/home-entertainment/',
 'tag/furniture/',
 'tag/cleaning/',
 'tag/cooling-heating/',
 'tag/laundry/',
 'tag/large-kitchen-appliances/',
 'tag/small-kitchen-appliances/',
 'tag/baking/',
 'tag/coffee/',
 'tag/smartphone/',
 'tag/headphones/',
 'tag/laptop/',
 'tag/networking/',
 'tag/computer/',
 'tag/accessories/']

### 2.0 Scraping links of pages with numbers

In [40]:
all_sections = [
    "reviews/",
    "tips/",
    "buying-guides-malaysia/",
    "tag/best-in-singapore/",
    "bm/"
]

In [41]:
# add the subsections from reviews
all_sections.extend(to_add)

In [42]:
additional_pages = []
for sections in all_sections:
    url = f"https://techrakyat.com/{sections}"
    response = requests.get(url, cookies=cookies, headers=headers)

    soup = BeautifulSoup(response.text, "lxml")

    block = soup.find('nav',attrs = {"class":"elementor-pagination"})
    
    if block is not None:
        for link in block.find_all('a', attrs = {"class":"page-numbers"}):
            try:
                href = link.get('href')
                page_nums = href.split(".")[-1][4:]
                additional_pages.append(page_nums)
            except:
                print(f"{sections.title()} doesn't have page numbers.")
    else:
        print(f"{sections.title()} doesn't have page numbers.")

Buying-Guides-Malaysia/ doesn't have page numbers.
Tag/Best-In-Singapore/ doesn't have page numbers.
Tag/Home-Entertainment/ doesn't have page numbers.
Tag/Furniture/ doesn't have page numbers.
Tag/Cleaning/ doesn't have page numbers.
Tag/Laundry/ doesn't have page numbers.
Tag/Baking/ doesn't have page numbers.
Tag/Coffee/ doesn't have page numbers.
Tag/Headphones/ doesn't have page numbers.
Tag/Networking/ doesn't have page numbers.
Tag/Computer/ doesn't have page numbers.


In [43]:
additional_pages

['reviews/page/2/',
 'reviews/page/3/',
 'reviews/page/4/',
 'reviews/page/5/',
 'reviews/page/2/',
 'tips/page/2/',
 'tips/page/3/',
 'tips/page/4/',
 'bm/page/2/',
 'bm/page/3/',
 'bm/page/4/',
 'bm/page/5/',
 'tag/cooling-heating/page/2/',
 'tag/cooling-heating/page/3/',
 'tag/large-kitchen-appliances/page/2/',
 'tag/large-kitchen-appliances/page/3/',
 'tag/small-kitchen-appliances/page/2/',
 'tag/small-kitchen-appliances/page/3/',
 'tag/small-kitchen-appliances/page/4/',
 'tag/smartphone/page/2/',
 'tag/smartphone/page/3/',
 'tag/smartphone/page/4/',
 'tag/laptop/page/2/',
 'tag/accessories/page/2/']

In [44]:
all_sections.extend(additional_pages)

In [45]:
all_sections.extend(["broadband-telco/"])

In [46]:
len(all_sections)

45

In [47]:
pages_unique = list(set(all_sections))
print(f'Num. of unique sections: {len(pages_unique)}')
with open(f'techrakyat_all_sections.json', 'a') as f:
    json.dump(pages_unique, f)

Num. of unique sections: 44


### 3.0 Get hrefs for each article

In [48]:
pages_unique = []
with open(f'techrakyat_all_sections.json') as fopen:
    links = json.load(fopen)
pages_unique.extend(links)

In [49]:
pages_unique

['reviews/page/4/',
 'tag/laundry/',
 'tag/large-kitchen-appliances/page/3/',
 'tag/smartphone/page/2/',
 'tag/smartphone/page/4/',
 'tag/cooling-heating/',
 'reviews/page/5/',
 'tag/large-kitchen-appliances/page/2/',
 'tag/networking/',
 'tag/computer/',
 'tag/cooling-heating/page/2/',
 'tag/cooling-heating/page/3/',
 'tag/smartphone/',
 'tag/laptop/page/2/',
 'buying-guides-malaysia/',
 'tag/best-in-singapore/',
 'bm/page/4/',
 'tag/accessories/',
 'tag/smartphone/page/3/',
 'bm/page/5/',
 'tag/small-kitchen-appliances/page/4/',
 'tag/furniture/',
 'reviews/page/3/',
 'broadband-telco/',
 'tag/home-entertainment/',
 'tag/laptop/',
 'tag/baking/',
 'tips/page/2/',
 'bm/page/3/',
 'tips/',
 'tag/cleaning/',
 'tag/small-kitchen-appliances/page/3/',
 'bm/',
 'tag/small-kitchen-appliances/',
 'tag/coffee/',
 'tag/headphones/',
 'bm/page/2/',
 'tag/small-kitchen-appliances/page/2/',
 'tag/large-kitchen-appliances/',
 'reviews/page/2/',
 'reviews/',
 'tips/page/3/',
 'tips/page/4/',
 'tag/a

In [50]:
def crawl(sections):
    while True:
        try:
            url = f"https://techrakyat.com/{sections}"
            response = requests.get(url, cookies=cookies, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(response.text, "lxml")

    block = soup.find('div', attrs = {"class":"site-content"})
    
    if block is None:
        return

    for link in block.find_all('h3', attrs = {"class":"elementor-post__title"}):
        try:
            href = link.find('a').get('href')
            hrefs.append(href)
        except:
            pass

In [51]:
block

<nav aria-label="Pagination" class="elementor-pagination">
<span aria-current="page" class="page-numbers current"><span class="elementor-screen-only">Page</span>1</span>
<a class="page-numbers" href="https://techrakyat.com/tag/accessories/page/2/"><span class="elementor-screen-only">Page</span>2</a> </nav>

In [52]:
max_worker = 10

hrefs = []
for i in tqdm(range(0, len(pages_unique), max_worker)):
    gc.collect()
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, t): t for t in pages_unique[i: i + max_worker]}

    for future in as_completed(futures):
        future.result()

print(f'Num. of raw links: {len(hrefs)}')
hrefs_unique = list(set(hrefs))
print(f'Num. of unique links: {len(hrefs_unique)}')
with open(f'techrakyat-article-links.json', 'a') as f:
    json.dump(hrefs_unique, f)

  0%|          | 0/5 [00:00<?, ?it/s]

Num. of raw links: 315
Num. of unique links: 221


### 4.0 Get text data from each article

In [53]:
# load articles that we need to scrape
articles = []
with open(f'techrakyat-article-links.json') as fopen:
    article_links = json.load(fopen)
articles.extend(article_links)

print(f"Num. of articles to scrape: {len(articles)}")

Num. of articles to scrape: 221


In [65]:
def crawl_article(url):
    while True:
        try:
            response = requests.get(url, cookies=cookies, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(response.text, "lxml")



    try:
        headline = soup.find('h1', class_="elementor-heading-title elementor-size-default").text
        words = soup.find('article', 
                          attrs = {"class":"elementor-section elementor-top-section elementor-element elementor-element-6513bcc elementor-section-boxed elementor-section-height-default elementor-section-height-default"}
                          ) # post-body entry-content
        content = words.text
        comments = soup.find('div', attrs={"class": "wpd-thread-list"}).text

    except Exception as e:
        print('error in link:'+ url)
        print(e)
        return None

    data = {
        'url': url, 
        'headline': headline, 
        'content': content, 
        'comments': comments
    }
    return data

In [68]:
# test
x = crawl_article("https://techrakyat.com/reviews/best-broadband-plan-malaysia/")
print(x)

{'url': 'https://techrakyat.com/reviews/best-broadband-plan-malaysia/', 'headline': 'Best Broadband Plan – Compare Unifi, Maxis & Time Fibre', 'content': '\n\n\n\n\n\n\n\nThe new Ministry of Communications and Multimedia had urged all broadband service providers to lower down their existing pricing and make high-speed internet available for ALL. The big three players in Malaysia, namely Unifi, Maxis Fibre and Time Fibre, responded\xa0by pushing out higher internet speed for existing subscribers and launching cheaper subscription tiers.\nThe ever-stagnant broadband market is now shaken up. Suddenly, you get news of turbo speed upgrade and new plans from service providers every now and then. At the same time, it also caught my attention to examine which one really offers the best broadband plan in Malaysia right now.\n\n\n\n\n\nTable of Contents\nToggle\nHow I pickThe analysis: Unifi vs. Maxis Fibre vs. Time FibreHow to choose the best broadband plan in MalaysiaQuick guide for a wifi rou

In [71]:
max_worker = 20

for i in tqdm(range(0, len(articles), max_worker)):
    gc.collect()
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl_article, t): t for t in articles[i: i + max_worker]}

    for future in as_completed(futures):
        result = future.result()
        if result:
            with open(f'techrakyat-scraped-data.jsonl', 'a') as final:
                json.dump(result, final)
                final.write('\n')

  0%|          | 0/12 [00:00<?, ?it/s]

error in link:https://techrakyat.com/bm/peti-sejuk-terbaik-malaysia/
'NoneType' object has no attribute 'text'
error in link:https://techrakyat.com/reviews/telco-plan-malaysia/
'NoneType' object has no attribute 'text'


In [77]:
"""
The second webpage's text content lies in the form of a table.
Hence, we'll have to ignore it.
"""

def crawl_last_article(url):
    while True:
        try:
            response = requests.get(url, cookies=cookies, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(response.text, "lxml")



    try:
        headline = soup.find('h1', class_="elementor-heading-title elementor-size-default").text
        words = soup.find('article', 
                          attrs = {"class":"elementor-section elementor-top-section elementor-element elementor-element-6513bcc elementor-section-boxed elementor-section-height-default elementor-section-height-default"}
                          ) # post-body entry-content
        content = words.text
        # comments = soup.find('div', attrs={"class": "wpd-thread-list"}).text

    except Exception as e:
        print('error in link:'+ url)
        print(e)
        return None

    data = {
        'url': url, 
        'headline': headline, 
        'content': content, 
        'comments': "" # there's no comments on that webpage anyway
    }
    return data

last_article = crawl_last_article("https://techrakyat.com/bm/peti-sejuk-terbaik-malaysia/")
print(last_article)

{'url': 'https://techrakyat.com/bm/peti-sejuk-terbaik-malaysia/', 'headline': '10 Peti Sejuk Terbaik Malaysia – 5 Bintang', 'content': '\n\n\n\n\n\n\n\nKini, kita boleh membeli pelbagai jenis peti sejuk di Malaysia. Antara model-model peti sejuk yang boleh didapati termasuk peti sejuk satu-pintu, 2-pintu, french door, dan peti ais mini.\nTerdapat juga peti sejuk pintar yang dilengkapi dengan skrin sentuh dan kamera, jadi pengguna boleh menyemak barang yang simpan dalam peti sejuk dengan menggunakan telefon pintar sahaja.\n\n\n\nUntuk mencari peti sejuk terbaik di Malaysia, kami telah menyelidik banyak jenama terkemuka peti sejuk seperti Panasonic, Midea, Toshiba, Midea, Hisense, Samsung dan LG.\n\n\nTable of Contents\nToggle\nApakah jenama peti sejuk terbaik di Malaysia?1. Toshiba GR-AG66MA2. Peti Seuk Panasonic 422L 2-Pintu3. Peti Sejuk 2-Pintu Midea MDRT268MTB284. Toshiba GR-A28MU5. Peti Sejuk Hisense RS666N4ACN 620L  6. Peti sejuk inverter dengan dispenser air Toshiba GR-RS637WE7. P

In [78]:
with open(f'techrakyat-scraped-data-fixed.jsonl', 'a') as final:
    json.dump(last_article, final)
    final.write('\n')

In [79]:
final_test = pd.read_json('techrakyat-scraped-data-fixed.jsonl', lines=True)
print(f"Num. of rows in dataframe: {len(final_test)}")

# check if there's any articles with no content
no_content = final_test[final_test['content'] == ""]
print(f"Num. of webpages with no content: {no_content}")
no_content.head(3)

Num. of rows in dataframe: 220
Num. of webpages with no content: Empty DataFrame
Columns: [url, headline, content, comments]
Index: []


,url,headline,content,comments
